***ETL TISS HOSPITALAR***

In [ ]:
###################instalando as bibliotecas
%pip install pandas 
%pip install requests
%pip install shutil
%pip install pandas sqlalchemy psycopg2

In [2]:
###################importando as bibliotecas 
import pandas as pd
import os
import requests
import shutil
from sqlalchemy import create_engine
from sqlalchemy import create_engine, text



In [ ]:
# 🔐 Conexão com o banco PostgreSQL
usuario = 'postgres'
senha = '7560221'
host = 'localhost'  # ou IP do servidor
porta = '5433'
banco = 'bases_cm'

# Cria engine de conexão
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}')

# 🧠 Função para executar comandos SQL
def executar_sql(comando_sql):
    with engine.connect() as conexao:
        conexao.execute(text(comando_sql))
        conexao.commit()


In [3]:
# 🔐 Conexão com o banco PostgreSQL (casa)
usuario = 'postgres'
senha = '7560221'
host = 'localhost'  # ou IP do servidor
porta = '5432'
banco = 'sis_datasus' #sis_datasus

# Cria engine de conexão
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}')

# 🧠 Função para executar comandos SQL
def executar_sql(comando_sql):
    with engine.connect() as conexao:
        conexao.execute(text(comando_sql))
        conexao.commit()

***01. PARAMETROS***

In [4]:
#############################PARAMETROS


############### ANOMES
anomes = (
'202301','202302','202303','202304','202305','202306','202307','202308','202309','202310','202311','202312',
'202401','202402','202403','202404','202405','202406','202407','202408','202409','202410','202411','202412'
)
anomes = pd.DataFrame (anomes)

############### UF
uf = (
"RO",
"AC",
"AM",
"RR",
"PA",
"AP",
"TO",
"MA",
"PI",
"CE",
"RN",
"PB",
"PE",
"AL",
"SE",
"BA",
"ES",
"PR",
"SC",
"RS",
"MS",
"MT",
"GO",
"DF",
"MG",
"RJ",
"SP"     
)
uf = pd.DataFrame (uf)




***02. HOSPITALAR***

*2.1 CONSOLIDADO*

In [ ]:
# Caminho base dos arquivos
#caminho_base = 'C:/Users/luis.lobato/OneDrive - fiosaude.org.br/01. BACKUP/02. EVENTOS FIOCRUZ/DTISS/HOSPITALAR/'
caminho_base = 'F:/DTISS/HOSPITALAR/'
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

# 🔁 Loop para abrir e carregar os arquivos
for i in range(0,n):      #anomes
    for j in range(0,m):  #uf
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]
        caminho_arquivo = os.path.join(caminho_base, uf_atual, f'{uf_atual}_{anomes_atual}_hosp_cons.csv')
        
        try:
            df = pd.read_csv(caminho_arquivo, sep=';', dtype=str)
            df.columns = [
"id_evento_atencao_saude",
"id_plano",
"faixa_etaria",
"sexo",
"cd_municipio_beneficiario",
"porte",
"cd_modalidade",
"nm_modalidade",
"cd_municipio_prestador",
"uf_prestador",
"tempo_de_permanencia",
"ano_mes_evento",
"cd_carater_atendimento",
"cd_tipo_internacao",
"cd_regime_internacao",
"cd_motivo_saida",
"cid_1",
"cid_2",
"cid_3",
"cid_4",
"qt_diaria_acompanhante",
"qt_diaria_uti",
"ind_acidente_doenca",
"lg_valor_preestabelecido"
            ]
            
            # 🧭 Define nome da tabela
            nome_tabela = f'{uf_atual.lower()}_{anomes_atual}_hosp_cons'
            
            # 🚀 Carrega no PostgreSQL
            df.to_sql(nome_tabela, engine, schema='dtiss2', if_exists='replace', index=False)
            print(f'Tabela {nome_tabela} carregada com sucesso.')
        
        except FileNotFoundError:
            print(f'Arquivo {uf_atual}_{anomes_atual} não encontrado.')
        except Exception as e:
            print(f'Erro ao carregar {uf_atual}_{anomes_atual}: {e}')



*2.2 DETALHADO*

In [ ]:
# Caminho base dos arquivos
#caminho_base = 'C:/Users/luis.lobato/OneDrive - fiosaude.org.br/01. BACKUP/02. EVENTOS FIOCRUZ/DTISS/HOSPITALAR/'
caminho_base = 'F:/DTISS/HOSPITALAR/'
# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

# 🔁 Loop para abrir e carregar os arquivos
for i in range(0,n):       # anomes
    for k in range(0,m):   # uf 
        uf_atual = uf.iloc[j,0]
        anomes_atual = anomes.iloc[i, 0]
        caminho_arquivo = os.path.join(caminho_base, uf_atual, f'{uf_atual}_{anomes_atual}_hosp_det.csv')
        
        try:
            df = pd.read_csv(caminho_arquivo, sep=';')
            df.columns = [
"id_evento_atencao_saude",
"uf_prestador",
"tempo_de_permanencia",
"ano_mes_evento",
"cd_procedimento",
"cd_tabela_referencia",
"qt_item_evento_informado",
"vl_item_evento_informado",
"vl_item_pago_fornecedor",
"unidade_medida",
"ind_pacote",
"ind_tabela_propria"
            ]
            
            # 🧭 Define nome da tabela
            nome_tabela = f'{uf_atual.lower()}_{anomes_atual}_hosp_det'
            
            # 🚀 Carrega no PostgreSQL
            df.to_sql(nome_tabela, engine, schema='dtiss2', if_exists='replace', index=False)
            print(f'Tabela {nome_tabela} carregada com sucesso.')
        
        except FileNotFoundError:
            print(f'Arquivo {uf_atual}_{anomes_atual} não encontrado.')
        except Exception as e:
            print(f'Erro ao carregar {uf_atual}_{anomes_atual}: {e}')

*2.3 FORMATANDO VARIAVEIS DO DETALHADO*

In [ ]:
n = anomes.shape[0]
m = uf.shape[0]


for i in range(0,n):       # anomes
    for k in range(0,m):   # uf 

        uf_str = uf.iloc[k, 0]
        anomes_str = anomes.iloc[i, 0]

        for tipo in ['det', 'cons']:
            tabela_base = f"dtiss2.{uf_str}_{anomes_str}_hosp_{tipo}"
            tabela_base1 = f"{uf_str}_{anomes_str}_hosp_{tipo}"
            tabela_temp = f"{tabela_base}_1"

            if tipo == 'det':
                campos = """
                    id_evento_atencao_saude,
                    uf_prestador,
                    tempo_de_permanencia,
                    ano_mes_evento,
                    substring(ano_mes_evento,1,4) as ano_realizacao,
                    cd_procedimento as cd_tuss_procedimento,
                    cd_tabela_referencia,
                    sum(COALESCE(NULLIF(REPLACE(qt_item_evento_informado::text, ',', '.'), '')::DECIMAL(1000,2), 0)) as QT_PROCEDIMENTO,
                    sum(COALESCE(NULLIF(REPLACE(vl_item_evento_informado::text, ',', '.'), '')::DECIMAL(1000,2), 0)) as VL_PROCEDIMENTO,
                    sum(COALESCE(NULLIF(REPLACE(vl_item_pago_fornecedor::text, ',', '.'), '')::DECIMAL(1000,2), 0)) as VL_PROCEDIMENTO_FORNECEDOR,
                    ind_pacote,
                    ind_tabela_propria
                """
                grupo = """
                    id_evento_atencao_saude,
                    uf_prestador,
                    tempo_de_permanencia,
                    ano_mes_evento,
                    ano_realizacao,
                    cd_procedimento,
                    cd_tabela_referencia,
                    ind_pacote,
                    ind_tabela_propria
                """
            else:  # cons
                campos = """
                    id_evento_atencao_saude,
                    id_plano,
                    faixa_etaria,
                    sexo,
                    cd_municipio_beneficiario,
                    porte,
                    cd_modalidade,
                    nm_modalidade,
                    uf_prestador,
                    tempo_de_permanencia,
                    ano_mes_evento,
                    substring(ano_mes_evento,1,4) as ano_realizacao,
                    cd_carater_atendimento,
                    cd_tipo_internacao,
                    cd_regime_internacao,
                    cd_motivo_saida,
                    cid_1,
                    cid_2,
                    cid_3,
                    cid_4,
                    sum(COALESCE(NULLIF(REPLACE(qt_diaria_acompanhante, ',', '.'), '')::DECIMAL(1000,2), 0)) as qt_diaria_acompanhante,
                    sum(COALESCE(NULLIF(REPLACE(qt_diaria_uti, ',', '.'), '')::DECIMAL(1000,2), 0)) as qt_diaria_uti,
                    ind_acidente_doenca,
                    lg_valor_preestabelecido
                """
                grupo = """
                    id_evento_atencao_saude,
                    id_plano,
                    faixa_etaria,
                    sexo,
                    cd_municipio_beneficiario,
                    porte,
                    cd_modalidade,
                    nm_modalidade,
                    uf_prestador,
                    tempo_de_permanencia,
                    ano_mes_evento,
                    ano_realizacao,
                    cd_carater_atendimento,
                    cd_tipo_internacao,
                    cd_regime_internacao,
                    cd_motivo_saida,
                    cid_1,
                    cid_2,
                    cid_3,
                    cid_4,
                    ind_acidente_doenca,
                    lg_valor_preestabelecido
                """

            # Criar nova tabela
            sql_create = f"CREATE TABLE {tabela_temp} AS SELECT {campos} FROM {tabela_base} GROUP BY {grupo}"
            executar_sql(sql_create)

            # Deletar tabela antiga
            sql_drop = f"DROP TABLE {tabela_base}"
            executar_sql(sql_drop)

            # Renomear nova tabela
            sql_rename = f"ALTER TABLE {tabela_temp} RENAME TO {tabela_base1}"
            executar_sql(sql_rename)

        print(f"✔️ Tabela DTISS_{uf_str}_{anomes_str} atualizada com sucesso.")

*2.4 MERGE BASE HOSPITALAR - DET E AMB AND SELECIONANDO PROCEDIMENTOS*

In [ ]:
###parametros
n = uf.shape[0]
m = anomes.shape[0]

for i in range(0,m):       # anomes
    for k in range(0,n):        # uf

        uf_str = uf.iloc[k, 0]
        anomes_str = anomes.iloc[i, 0]

        tabela_final = f"dtiss2.{uf_str}_hosp_{anomes_str}"
        tabela_det = f"dtiss2.{uf_str}_{anomes_str}_hosp_det"
        tabela_cons = f"dtiss2.{uf_str}_{anomes_str}_hosp_cons"

        sql = f"""
    CREATE TABLE {tabela_final} AS
SELECT
    det.*,
    cons.id_plano,
    cons.faixa_etaria,
    cons.sexo,
    cons.cd_municipio_beneficiario,
    cons.porte,
    cons.cd_modalidade,
    cons.nm_modalidade,
    cons.cd_carater_atendimento,
    cons.cd_tipo_internacao,
    cons.cd_regime_internacao,
    cons.cd_motivo_saida,
    cons.cid_1,
    cons.cid_2,
    cons.cid_3,
    cons.cid_4,
    cons.qt_diaria_acompanhante,
    cons.qt_diaria_uti,
    cons.ind_acidente_doenca,
    cons.lg_valor_preestabelecido
FROM {tabela_det} det
INNER JOIN {tabela_cons} cons
    ON CAST(det.id_evento_atencao_saude AS TEXT) = CAST(cons.id_evento_atencao_saude AS TEXT)
   AND det.uf_prestador = cons.uf_prestador
   AND det.ano_mes_evento = cons.ano_mes_evento
   AND det.ano_realizacao = cons.ano_realizacao
        """
        executar_sql(sql)
        print(f"✔️ Tabela DTISS_{uf_str}_{anomes_str} criada com sucesso.")

In [ ]:
# Criando as tabelas finais por UF

# Parâmetros
n = uf.shape[0]

for k in range(n):  # Itera sobre cada UF
    uf_atual = uf.iloc[k, 0]

    # Definindo o nome da tabela
    nome_tabela = f'dtiss2.{uf_atual}_hosp'

    # Estrutura da tabela
    estrutura_tabela = '''
        id_evento_atencao_saude INT8 NULL,
        uf_prestador TEXT NULL,
        tempo_de_permanencia TEXT NULL,
        ano_mes_evento TEXT NULL,
        ano_realizacao TEXT NULL,
        cd_tuss_procedimento TEXT NULL,
        cd_tabela_referencia INT8 NULL,
        qt_procedimento NUMERIC NULL,
        vl_procedimento NUMERIC NULL,
        vl_procedimento_fornecedor NUMERIC NULL,
        ind_pacote INT8 NULL,
        ind_tabela_propria INT8 NULL,
        id_plano TEXT NULL,
        faixa_etaria TEXT NULL,
        sexo TEXT NULL,
        cd_municipio_beneficiario TEXT NULL,
        porte TEXT NULL,
        cd_modalidade TEXT NULL,
        nm_modalidade TEXT NULL,
        cd_carater_atendimento TEXT NULL,
        cd_tipo_internacao TEXT NULL,
        cd_regime_internacao TEXT NULL,
        cd_motivo_saida TEXT NULL,
        cid_1 TEXT NULL,
        cid_2 TEXT NULL,
        cid_3 TEXT NULL,
        cid_4 TEXT NULL,
        qt_diaria_acompanhante NUMERIC NULL,
        qt_diaria_uti NUMERIC NULL,
        ind_acidente_doenca TEXT NULL,
        lg_valor_preestabelecido TEXT NULL
    '''

    # Comando SQL completo
    sql = f'CREATE TABLE {nome_tabela} ({estrutura_tabela})'

    # Executa a criação da tabela
    executar_sql(sql)

    # Confirmação
    print(f'✔️ Tabela {nome_tabela} criada com sucesso.')

In [ ]:
# 🔄 Juntando as tabelas mensais por UF

# Parâmetros
n = anomes.shape[0]
m = uf.shape[0]

for i in range(0,n):      # Itera sobre cada ano-mês
    for j in range(0,m):    # Itera sobre cada UF
        uf_atual = uf.iloc[j, 0]
        anomes_atual = anomes.iloc[i, 0]

        # Nome da tabela de origem e destino
        tabela_origem = f'dtiss2.{uf_atual}_hosp_{anomes_atual}'
        tabela_destino = f'dtiss2.{uf_atual}_hosp'

        # Comando SQL para inserção
        sql = f'''
            INSERT INTO {tabela_destino}
            SELECT * FROM {tabela_origem}
        '''

        # Executa o comando SQL
        executar_sql(sql)

        # Confirmação visual
        print(f'✔️ Dados de {tabela_origem} inseridos em {tabela_destino} com sucesso.')

*2.5 CRIANDO A TABELA DTISS BRASIL - BR*

In [8]:
#####CRIANDO AS TABELAS BR

##juntando as tabelas com procedimentos selecionados
s1 = f'CREATE TABLE dtiss2.br_hosp ('
s2 = '''
 	id_evento_atencao_saude int8 NULL,
	uf_prestador text NULL,
	tempo_de_permanencia text NULL,
	ano_mes_evento text NULL,
	ano_realizacao text NULL,
	cd_tuss_procedimento text NULL,
	cd_tabela_referencia int8 NULL,
	qt_procedimento numeric NULL,
	vl_procedimento numeric NULL,
	vl_procedimento_fornecedor numeric NULL,
	ind_pacote int8 NULL,
	ind_tabela_propria int8 NULL,
	id_plano text NULL,
	faixa_etaria text NULL,
	sexo text NULL,
	cd_municipio_beneficiario text NULL,
	porte text NULL,
	cd_modalidade text NULL,
	nm_modalidade text NULL,
	cd_carater_atendimento text NULL,
	cd_tipo_internacao text NULL,
	cd_regime_internacao text NULL,
	cd_motivo_saida text NULL,
	cid_1 text NULL,
	cid_2 text NULL,
	cid_3 text NULL,
	cid_4 text NULL,
	qt_diaria_acompanhante numeric NULL,
	qt_diaria_uti numeric NULL,
	ind_acidente_doenca text NULL,
	lg_valor_preestabelecido text NULL
);
'''
sql = s1 + s2
executar_sql (sql)  

In [10]:
# 🧬 Alimentando a tabela mãe com as tabelas filhas (herança PostgreSQL)

# Parâmetros
m = uf.shape[0]

for j in range(0, m):  # Itera sobre cada UF
    uf_atual = uf.iloc[j, 0]
    # Comando SQL para herdar da tabela mãe
    sql = f'''
        ALTER TABLE dtiss2.{uf_atual}_hosp
        INHERIT dtiss2.br_hosp
    '''    # Executa o comando
    executar_sql(sql)
    # Confirmação visual
    print(f'✔️ Tabela dtiss2.{uf_atual}_hosp agora herda de dtiss2.br_hosp com sucesso.')

✔️ Tabela dtiss2.RO_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.AC_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.AM_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.RR_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.PA_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.AP_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.TO_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.MA_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.PI_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.CE_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.RN_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.PB_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.PE_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.AL_hosp agora herda de dtiss2.br_hosp com sucesso.
✔️ Tabela dtiss2.SE_hosp agora herda de dtiss2.b

*2.6 FREQUENCIA*

In [ ]:
#custo de internação por regime de internação
sintaxe = '''
create table dtiss2.br_hosp_RI as 
select cd_regime_internacao , ano_realizacao , sum (vl_procedimento) as VL_PROCEDIMENTO
FROM dtiss2.br_hosp
WHERE
id_evento_atencao_saude <> '9763517'
group by cd_regime_internacao, ano_realizacao
order by  ano_realizacao,cd_regime_internacao;
'''
sql = ( sintaxe )
executar_sql (sql)


#custo de internação por modalidade de operadora
sintaxe = '''
create table dtiss2.br_hosp_MO as 
SELECT cd_REGIME_INTERNACAO,CD_MODALIDADE, NM_MODALIDADE , ano_realizacao,
sum (vl_procedimento) as VL_PROCEDIMENTO 
FROM dtiss2.br_hosp
WHERE 
id_evento_atencao_saude <> '9763517' 
group by  cd_REGIME_INTERNACAO,CD_MODALIDADE, NM_MODALIDADE, ano_realizacao
order by  
cd_REGIME_INTERNACAO asc,CD_MODALIDADE asc, NM_MODALIDADE asc, ano_realizacao asc;

'''
sql = ( sintaxe )
executar_sql (sql)


print(["✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------"] ) 

['✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------']


In [5]:

#--------------------------------------------retirando os duplicados da internacao
#------identificando os duplicados e retirando
sintaxe = '''
create table dtiss2.br_hosp_semdupl as
SELECT id_evento_atencao_saude,cd_REGIME_INTERNACAO,CD_MODALIDADE, NM_MODALIDADE , ano_realizacao,
  ROW_NUMBER() OVER(PARTITION by id_evento_atencao_saude ORDER BY id_evento_atencao_saude asc) AS duplicado,
 VL_PROCEDIMENTO 
FROM dtiss2.br_hosp
WHERE 
id_evento_atencao_saude <> '9763517' and
ano_realizacao in ('2023','2024')
ORDER BY 
 id_evento_atencao_saude asc,cd_REGIME_INTERNACAO asc,CD_MODALIDADE asc, NM_MODALIDADE asc, 
 ano_realizacao asc,VL_PROCEDIMENTO desc
'''
sql = ( sintaxe )
executar_sql (sql)

sintaxe = '''
create table dtiss2.br_hosp_semdupl_2 as
select *from dtiss2.br_hosp_semdupl
where 
 duplicado <= 1
 '''
sql = ( sintaxe )
executar_sql (sql)


#-- quantidade de internação por regime de internação
sintaxe = '''
create table dtiss2.br_hosp_semdupl_RI as 
select cd_regime_internacao , ano_realizacao , count (id_evento_atencao_saude) as N_internacao
FROM dtiss2.br_hosp_semdupl_2
WHERE
id_evento_atencao_saude <> '9763517'
group by cd_regime_internacao, ano_realizacao
order by  ano_realizacao,cd_regime_internacao
'''
sql = ( sintaxe )
executar_sql (sql)


#-- quantidade de internação por modalidade de operadora
sintaxe = '''
create table dtiss2.br_hosp_semdupl_MO as 
SELECT cd_REGIME_INTERNACAO,CD_MODALIDADE, NM_MODALIDADE , ano_realizacao,
count (id_evento_atencao_saude) as N_internacao 
FROM dtiss2.br_hosp_semdupl_2
WHERE 
id_evento_atencao_saude <> '9763517' 
group by  cd_REGIME_INTERNACAO,CD_MODALIDADE, NM_MODALIDADE, ano_realizacao
order by  
cd_REGIME_INTERNACAO asc,CD_MODALIDADE asc, NM_MODALIDADE asc, ano_realizacao asc
'''
sql = ( sintaxe )
executar_sql (sql)

print(["✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------"] ) 

['✔️---------TABELA DTISS AGREGADA CRIADA com sucesso ---------']
